In [1]:
import cv2
import os
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

# Creating dataset

Read files

In [2]:
files = os.listdir('Data')
dimension = 1250 # to powinno się dać jakoś automatycznie wczytać.
data = np.zeros((dimension, dimension, len(files)))
bands_names = []

In [3]:
for i, file in enumerate(files):
    bands_names.append(file[-7:-4])
    data[:, :, i] = cv2.imread(os.path.join('Data', file), cv2.IMREAD_UNCHANGED)[:dimension, :dimension]
X = data.reshape(dimension*dimension, len(files))

Dealing with outstanding data $ x = min(x,\overline{x}+3\sigma_{x}) $
* TODO plot with min max and avg for all/few bands to show why we have to do this

In [4]:
for i in range(X.shape[1]):
    X[:, i] = np.where(X[:, i] < np.mean(X[:, i]) + 3*np.std(X[:, i]), X[:, i], np.mean(X[:, i]) + 3*np.std(X[:, i]))

Normalisation
* TO DO: Normalisation should be done on training and test set seperatly

In [5]:
sc = MinMaxScaler(feature_range=(0, 1))
X = sc.fit_transform(X)

Read classes from Classification.png

In [6]:
Y_real = cv2.imread('Classification.png')[:dimension, :dimension]
Y_real = Y_real.reshape((dimension*dimension, 3))
Y_real = (Y_real/255).astype(int)
Other = (1 - Y_real.any(axis=1).astype(int)).reshape(-1,1)

Saving data to csv file

In [7]:
df = pd.DataFrame(np.concatenate((X, Y_real, Other), axis=1),
                  columns=bands_names + ['water', 'forest', 'fields', 'Other'])
df.to_csv('data.csv')

In [8]:
%reset -f

# Supervised classification

### Importin the libraries

In [9]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier

In [10]:
def undersampling(df):
# TODO Make it reusable
    forests = df[df["forest"]==1]
    water = df[df["water"]==1]
    fields = df[df["fields"]==1]
    other = df[df["Other"]==1]
    min_len = min(forests.shape[0], water.shape[0], fields.shape[0], other.shape[0])
    forests = forests.sample(min_len)
    water = water.sample(min_len)
    fields = fields.sample(min_len)
    other = other.sample(min_len)
    return water.append([fields,forests, other])

### Preprocessing

In [22]:
df = pd.read_csv('data.csv')
df = undersampling(df)
X = df.iloc[:,1:14].to_numpy()
Y = df.iloc[:,-4:].to_numpy()

Train/test split

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3)

Feature scaling
$$x = \frac{\overline{x} - x}{\sigma_{x}}$$

In [18]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

 Dimensionlity reduction: Principal component analysis
 13 -> 10
It speedup learning proces but decrease accuracy

In [19]:
# Applying PCA
from sklearn.decomposition import PCA
pca = PCA(n_components = 10)
X_train = pca.fit_transform(X_train)
X_test = pca.transform(X_test)

TO DO Cross Validation

### Training

In [20]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier()
clf.fit(X_train, y_train)

RandomForestClassifier()

### Validation

In [21]:
from sklearn.metrics import confusion_matrix, accuracy_score
y_pred = clf.predict(X_test)
cm = confusion_matrix(y_test.argmax(axis=1), y_pred.argmax(axis=1))
print(cm)
accuracy_score(y_test, y_pred)

[[16384   872    11   130]
 [  854 15175    53   942]
 [  479    20 14126  2474]
 [ 1245  1841  2983 11327]]


0.8229874049567589

# Unsupervised classification with SOM

In [23]:
def IoU(target, predicted):
    """return intersection over union"""
    iou = np.sum(np.logical_and(target,predicted))
    iou = iou / (np.sum(np.logical_or(target,predicted)) + 1e-10)
    return iou

In [24]:
def metrics_matrix(Y_target, Y_pred, metric=IoU):
    """Compare target classes with predicted classes. 
    Return matrix with metrics
    """
    matrix = np.zeros((Y_pred.shape[1],Y_target.shape[1]))
    for i in range(matrix.shape[0]):
        for j in range(matrix.shape[1]):    
            matrix[i][j] = IoU(Y_target[:,j],Y_pred[:,i])
    return matrix